In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np 
import pandas as pd 
import collections
import torch
import feather
import json
import os
import cv2
import random
import gc
import pycocotools

from tqdm import tqdm
import matplotlib.pyplot as plt
import PIL
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
from environs import Env

from detectron2 import model_zoo
from detectron2.structures import BoxMode
from detectron2.engine import DefaultPredictor, default_argument_parser, default_setup
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import setup_logger

import sys
sys.path.append('../')

from iMaterialist2020.imaterialist.data.datasets.coco import register_datadict
from iMaterialist2020.imaterialist.config import add_imaterialist_config
from iMaterialist2020.imaterialist.modeling import build_model

env = Env()
env.read_env()

In [3]:
# Get training dataframe
data_dir = Path(env('path_raw'))
image_dir = Path(env('path_images'))
df = pd.read_csv(data_dir/'train.csv')

# Load modified df for Detectron2 dataset dict 
df_detectron = pd.read_feather('../data/interim/imaterialist_train_multihot_n=4000.feather') 

# Get label descriptions
with open(data_dir/'label_descriptions.json', 'r') as file:
    label_desc = json.load(file)
df_categories = pd.DataFrame(label_desc['categories'])
df_attributes = pd.DataFrame(label_desc['attributes'])

In [4]:

df_detectron.ImageId[0]

'/home/nasty/imaterialist2020/data/raw/train/00000663ed1ff0c4e0132b9b9ac53f6e.jpg'

In [5]:
plt.imshow(df_detectron.ImageId[0])

TypeError: Image data of dtype <U80 cannot be converted to float

In [34]:
def setup(args):
    cfg = get_cfg()
    add_imaterialist_config(cfg)
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.merge_from_file(args.config_file)
    
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    # Setup logger for "imaterialist" module
    setup_logger(output=cfg.OUTPUT_DIR, distributed_rank=comm.get_rank(), name="imaterialist")
    return cfg

In [40]:
class iMatPredictor(DefaultPredictor):
    def __init__(self, cfg):
        #super().__init__(cfg)
        self.model = build_model(self, cfg)


In [41]:


datadic_train = pd.read_feather(path_data_interim / 'imaterialist_train_multihot_n=4000.feather')
datadic_val = pd.read_feather(path_data_interim / 'imterailist_val_multihot_n=1000.feather')
register_datadict(datadic_train, "sample_fashion_train")
register_datadict(datadic_val, "sample_fashion_test")

cfg = setup(args)

im = cv2.imread(df_detectron.ImageId[0])

predictor = iMatPredictor(im)

In [39]:
args = default_argument_parser().parse_args()
args.config_file = "/home/nasty/imaterialist2020/iMaterialist2020/configs/exp01.yaml"
main(args)

usage: ipykernel_launcher.py [-h] [--config-file FILE] [--resume]
                             [--eval-only] [--num-gpus NUM_GPUS]
                             [--num-machines NUM_MACHINES]
                             [--machine-rank MACHINE_RANK]
                             [--dist-url DIST_URL]
                             ...
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2